In [ ]:
import copy
from typing import Literal

class GuardTracker:
    guard_position: dict = {'y':0, 'x':0}
    guard_direction: Literal['up', 'right', 'down', 'left'] = 'up'
    file_name: str
    map: list[list[str]]
    visited = set()
    distinct_visited = set()
    forbidden_obstacle = {'y': 0, 'x': 0}
    print_out = True
    
    def __init__(self, file_name, extra_obstacle=None):
        self.file_name = file_name
        self._init_map(file_name, extra_obstacle)

                        
    def _init_map(self, file_name, extra_obstacle):
        with open(file_name) as f:
            map_rows_raw = list(f.readlines())
            self.map = [x[:len(x)-1] for x in [list(x) for x in map_rows_raw]]
            break_flag = False
            for y, row in enumerate(self.map):
                if break_flag:
                    break
                for x, elem in enumerate(row):
                    if elem == '^':
                        self.guard_position['x'] = x
                        self.guard_position['y'] = y
                        self.forbidden_obstacle = {'y': y, 'x': x}
                        break_flag = True
                        break
            if extra_obstacle:
                if not (extra_obstacle.get('y') == self.forbidden_obstacle.get('y') and extra_obstacle.get('x') == self.forbidden_obstacle.get('x')):
                    self.map[extra_obstacle.get('y')][extra_obstacle.get('x')] = '#'
                        
    def reset_with(self, file_name, extra_obstacle):
        self.visited = set()
        self.distinct_visited = set()
        self.guard_direction = 'up'
        self._init_map(file_name, extra_obstacle)
                        
    
    def _draw_map(self):
        self.map[self.guard_position.get('y')][self.guard_position.get('x')] = '&'
        for line in self.map:
            print("".join(line))
        self.map[self.guard_position.get('y')][self.guard_position.get('x')] = 'X'
            
    
    def _move_guard(self, map, guard_position, guard_direction, look_for_loop=False):
        if guard_direction == 'up':
            if guard_position.get('y') == 0:
                return True, guard_direction
            elif map[guard_position.get('y') - 1][guard_position.get('x')] == '#':
                guard_direction = 'right'
                return False, guard_direction
            else:
                guard_position['y'] -= 1
                return False, guard_direction
        elif guard_direction == 'down':
            if guard_position.get('y') == len(map) - 1:
                return True, guard_direction
            elif map[guard_position.get('y') + 1][guard_position.get('x')] == '#':
                guard_direction = 'left'
                return False, guard_direction
            else:
                guard_position['y'] += 1
                return False, guard_direction
        elif guard_direction == 'right':
            if guard_position.get('x') == len(map[0]) - 1:
                return True, guard_direction
            elif map[guard_position.get('y')][guard_position.get('x') + 1] == '#':
                guard_direction = 'down'
                return False, guard_direction
            else:
                guard_position['x'] += 1
                return False, guard_direction
        else:
            if guard_position.get('x') == 0:
                return True, guard_direction
            elif map[guard_position.get('y')][guard_position.get('x') - 1] == '#':
                guard_direction = 'up'
                return False, guard_direction
            else:
                guard_position['x'] -= 1
                return False, guard_direction
            
    def _count_distinct(self):
        for i in self.visited:
            dist = "-".join(i.split('-')[:2])
            self.distinct_visited.add(dist)
        if self.print_out:
            print("visited distinct", len(self.distinct_visited))
            
    def start_walking(self, print_out=True):
        self.print_out = print_out
        walk_counter = 0
        while True:
            went_out, self.guard_direction = self._move_guard(self.map, self.guard_position, self.guard_direction)
            if went_out:
                break
            description = f'{self.guard_position.get('y')}-{self.guard_position.get('x')}-{self.guard_direction}'
            if description in self.visited:
                return 1
            self.visited.add(f'{self.guard_position.get('y')}-{self.guard_position.get('x')}-{self.guard_direction}')
            walk_counter += 1
        self._count_distinct()
        return 0
    
    def get_distinct(self):
        return self.distinct_visited


In [ ]:
guardTracker = GuardTracker('map.txt')
guardTracker.start_walking()

In [ ]:
look_at_these = set(guardTracker.get_distinct())

In [ ]:
len(look_at_these)

In [ ]:
loops = 0

for look in look_at_these:
    guardTracker.reset_with('map.txt', {'y': int(look.split('-')[0]), 'x': int(look.split('-')[1])})
    loops += guardTracker.start_walking(False)

loops